## EyeWatchOne Engineering Prototype

Hyper local weather related accident prediction.

Gives a browser/mobile based heatmap overlay on satellite images showing where accidents are predicted to occur given historical accidents and incoming weather.


Targeted towards government planning for road emergency services, but can also be used for insurance claims adjustment, logistics companies's route planning, and much more. 


### Move Historical Insurance Data to S3

code adapted from 
http://boto.cloudhackers.com/en/latest/s3_tut.html

fix .py file association error
http://superuser.com/questions/429604/passing-arguments-to-a-python-script-file-association-not-found-windows-7-on-i

In [10]:
import os
import sys
os.path.dirname(sys.executable)

'C:\\Users\\HN\\Anaconda2'

In [11]:
! aws s3 ls

2016-02-27 14:52:54 eyewatchone


### Upload test entry to S3 bucket: eyewatchone

In [29]:
! aws s3 cp data/example_compressed_entry.zip s3://eyewatchone --grants read=uri=http://acs.amazonaws.com/groups/global/AllUsers full=emailaddress=hollisnolan@gmail.com

upload: data/example_compressed_entry.zip to s3://eyewatchone/example_compressed_entry.zip


In [32]:
! aws s3 cp data/dictionary.html s3://eyewatchone --grants read=uri=http://acs.amazonaws.com/groups/global/AllUsers full=emailaddress=hollisnolan@gmail.com

upload: data/dictionary.html to s3://eyewatchone/dictionary.html


In [6]:
! aws s3 cp data/test_set.zip s3://eyewatchone --grants read=uri=http://acs.amazonaws.com/groups/global/AllUsers full=emailaddress=hollisnolan@gmail.com

upload: data/test_set.zip to s3://eyewatchone/test_set.zip


In [7]:
! aws s3 cp data/train_set.zip s3://eyewatchone --grants read=uri=http://acs.amazonaws.com/groups/global/AllUsers full=emailaddress=hollisnolan@gmail.com

upload: data/train_set.zip to s3://eyewatchone/train_set.zip


In [12]:
! aws s3 ls s3://eyewatchone

2016-02-27 18:34:54      24768 dictionary.html
2016-02-27 18:27:26    5120252 example_compressed_entry.zip
2016-02-27 20:29:50  138124104 test_set.zip
2016-02-27 20:31:27  380553491 train_set.zip


### Spin up analytics EC2 instance

(right click for paste in windows)

In [ ]:
# create an instance, DS-toolbox ami and XL size

!aws ec2 run-instances --image-id ami-d1737bb8 --count 1 --instance-type m3.xlarge --key-name .boto --security-groups my-sg

In [ ]:
ec2-attach-volume snap-ac47f4c5 --instance i-582bf5dc --device  /dev/sdf

In [ ]:
# ssh in (in terminal) 
ssh -X -i ~/.ssh/aws.pem ubuntu@ec2-52-90-250-206.compute-1.amazonaws.com

# update 
sudo apt-get update && sudo apt-get upgrade

# update pip 
sudo pip install pip --upgrade

# install notebook
sudo pip install --upgrade "ipython[notebook]"

# open ports 
In the Security Group, select Inbound, then Edit, and then Add Rule. The Port Range should be 8888 and the Sourceis 0.0.0.0/0

# start tmux 
tmux new -s notebook

# new notebook with ip specified
jupyter notebook --no-browser --ip=0.0.0.0

# grab ip address of instance and navigate to 
http://52.90.250.206:8888/tree
        
# save this notebook, close it locally, and use the "upload" button to move it into the aws instance
# rerun notebook from there

In [17]:
# restart a stopped instance 

!aws ec2 start-instances --instance-ids i-582bf5dc --region us-east-1

{
    "StartingInstances": [
        {
            "InstanceId": "i-582bf5dc", 
            "CurrentState": {
                "Code": 0, 
                "Name": "pending"
            }, 
            "PreviousState": {
                "Code": 80, 
                "Name": "stopped"
            }
        }
    ]
}


### Check historical weather in EBS public data set

how to 
http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ebs-attaching-volume.html

public weather
https://aws.amazon.com/datasets/daily-global-weather-measurements-1929-2009-ncdc-gsod/

List disks 

In [ ]:
lsblk

mount disk 
(help from)
https://help.ubuntu.com/community/InstallingANewHardDrive

In [ ]:
sudo mount /dev/xvdf /hist_weather/

In [ ]:
sudo pip install awscli

In [ ]:
aws configure 

###  Move historical insurance to EBS

In [ ]:
aws s3 ls s3://eyewatchone

In [ ]:
sudo mkdir hist_insurance

In [ ]:
sudo aws s3 cp s3://eyewatchone/example_compressed_entry.zip hist_insurance/

In [ ]:
# mistake, should be 
sudo aws s3 cp s3://eyewatchone/example_compressed_entry.zip analytics_ebs/

In [ ]:
sudo unzip hist_insurance/example_compressed_entry.zip

###  Move a sample image to EBS

In [ ]:
aws s3 ls s3://nasanex/Landsat

### Query Weather API, Store to EBS

code adapted from http://stackoverflow.com/questions/12965203/how-to-get-json-from-webpage-into-python-script
and https://www.wunderground.com/weather/api/d/docs?MR=1

In [8]:
import urllib, json
from pprint import pprint

url = "http://api.wunderground.com/api/      api-key       /conditions/q/CA/San_Francisco.json"
response = urllib.urlopen(url)
data = json.loads(response.read())
pprint(data)

{u'current_observation': {u'UV': u'4',
                          u'dewpoint_c': 10,
                          u'dewpoint_f': 50,
                          u'dewpoint_string': u'50 F (10 C)',
                          u'display_location': {u'city': u'San Francisco',
                                                u'country': u'US',
                                                u'country_iso3166': u'US',
                                                u'elevation': u'47.00000000',
                                                u'full': u'San Francisco, CA',
                                                u'latitude': u'37.77500916',
                                                u'longitude': u'-122.41825867',
                                                u'magic': u'1',
                                                u'state': u'CA',
                                                u'state_name': u'California',
                                                u'wmo': u'99999',
     

Store to EBS

### Produce a Prediction

### Tag the Image, Send to Team

### Clone the Analytics EC2, Scale for Production

### Kafka, batch up Weather Stream?